## algorithm design and anlysis-2025 spring  homework 3
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1  

请证明基于比较的排序算法计算复杂度下界为$\Omega(nlogn)$

you can refer a handwritten picture, pleas upload the picture in /fig/xxx.png
answer：<div style="text-align:center;"> <img src="./fig/hw3q1.jpg" width="800"/> </div>

## 问题 2 

构建二叉搜索树

> 1. 给定一个数组 [48, 33, 49, 47, 42, 46, 32], 构建一个二叉搜索树存储这些数据，请**绘制**所构建的**二叉搜索树**（标明结点的值）。
> 2. 从第1问的二叉搜索树中删除33，请**绘制**删除33后的**二叉搜索树**（标明结点的值）

- 推荐使用graphviz包绘制树的结构

you can refer a handwritten picture, pleas upload the picture in /fig/xxx.png
answer：<div style="text-align:center;"> <img src="./fig/hw3q2.jpg" width="800"/> </div>

## 问题 3 

> 下面是九门课程的开始和结束时间:
>
> [(9:00,12:30), (11:00,14:00), (13:00, 14:30), (9:00,10:30),(13:00, 14:30),(14:00,16:30), (15:00,16:30), (15:00,16:30), (9:00,10:30)]
>
> 请使用贪婪算法为这九门课分配教室，要求在同一天内安排这些课，并且每个教室同一时间只能安排一门课。

**请问最少需要几间教室，罗列出每个教室安排的课程**

idea：首先将所有课程按开始时间升序排列，如果开始时间相同则按结束时间升序排列。这确保我们总是处理当前最早的课程。然后使用最小堆来跟踪每个教室的当前结束时间。每次处理课程时，选择结束时间最早的教室。如果该教室的结束时间不晚于当前课程的开始时间，则分配该课程到该教室，否则需要新的教室。最后记录每个教室分配的课程，并实时更新它们的结束时间。堆中存储各教室的结束时间和索引，以便快速获取最早结束的教室

In [1]:
# code：
import heapq

def time_to_minutes(t):
    # 将时间字符串转换为分钟数
    h, m = map(int, t.split(":"))
    return h * 60 + m

def minutes_to_time(m):
    # 将分钟数转换为时间字符串
    return f"{m // 60:02d}:{m % 60:02d}"

 # 根据课程时间安排最少教室数量。
def min_classrooms(courses):
   
    # 转换为分钟并排序
    intervals = [(time_to_minutes(s), time_to_minutes(e)) for s, e in courses]
    intervals.sort()

    classrooms = []           # 每间教室的信息，包含课程与结束时间
    min_heap = []             # 存储当前教室的最早结束时间：(end_time, classroom_index)

    for start, end in intervals:
        if min_heap and min_heap[0][0] <= start:
            # 如果有教室空闲，将课程安排到该教室
            earliest_end, idx = heapq.heappop(min_heap)
            classrooms[idx]["courses"].append((start, end))
            classrooms[idx]["end_time"] = end
            heapq.heappush(min_heap, (end, idx))
        else:
            # 否则，新增一间教室
            new_classroom = {
                "courses": [(start, end)],
                "end_time": end
            }
            classrooms.append(new_classroom)
            heapq.heappush(min_heap, (end, len(classrooms) - 1))

    # 整理输出
    result = []
    for room in classrooms:
        formatted_courses = [
            f"({minutes_to_time(s)}, {minutes_to_time(e)})"
            for s, e in room["courses"]
        ]
        result.append(formatted_courses)

    return result

def main():
    input_courses = [
        ("9:00", "12:30"), ("11:00", "14:00"), ("13:00", "14:30"),
        ("9:00", "10:30"), ("13:00", "14:30"), ("14:00", "16:30"),
        ("15:00", "16:30"), ("15:00", "16:30"), ("9:00", "10:30")
    ]

    assignments = min_classrooms(input_courses)
    print(f"最少需要 {len(assignments)} 间教室，具体安排如下：\n")
    for i, room_courses in enumerate(assignments, 1):
        print(f"教室 {i}:")
        for course in room_courses:
            print(f"  {course}")
        print()

if __name__ == "__main__":
    main()



最少需要 3 间教室，具体安排如下：

教室 1:
  (09:00, 10:30)
  (11:00, 14:00)
  (14:00, 16:30)

教室 2:
  (09:00, 10:30)
  (13:00, 14:30)
  (15:00, 16:30)

教室 3:
  (09:00, 12:30)
  (13:00, 14:30)
  (15:00, 16:30)



## 问题 4 

> 爬楼梯问题：假设爬楼梯时你每次只能爬一阶或者爬两阶，问爬上n阶的楼梯，你一共有多少种方法

请设计算法完成该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：到达第 n 阶的方法数等于从第 n-1 阶爬一步（方法数继承自 n-1）和从第 n-2 阶爬两步（方法数继承自 n-2）的总和，递推公式为 f(n) = f(n-1) + f(n-2)

In [3]:
# add your code here
def climb_stairs(n: int) -> int:
    if n <= 2:
        return n
    prev, curr = 1, 2
    for _ in range(3, n + 1):
        prev, curr = curr, prev + curr
    return curr

if __name__ == "__main__":
    test_cases = [1, 2, 3, 4, 5]
    for n in test_cases:
        print(f"n={n}时，方法数：{climb_stairs(n)}")

# your algorithm time complexity is: O(n)

n=1时，方法数：1
n=2时，方法数：2
n=3时，方法数：3
n=4时，方法数：5
n=5时，方法数：8


## 问题 5 

> 0-1背包问题：现在有4块大理石，每个大理石对应的重量和价值使用一个元组表示, 即（重量，价值），4块大理石的重量和价值为：[(5,10), (4,40), (6,30), (3, 50)], 假设你有一辆最大承重为9的小推车，请问使用这个小推车装大理石的最大价值为多少

请设计一个算法解决该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：0-1背包问题的核心是决策每个物品是否放入背包以最大化总价值。使用动态规划解决该问题，定义一维数组 dp[j] 表示承重为 j 时的最大价值。逆序遍历每个物品（重量 w，价值 v），从 max_weight 到 w 更新 dp[j] = max(dp[j], dp[j - w] + v)，确保每个物品仅被计算一次，最终 dp[max_weight] 即为最大价值。

In [2]:
# add your code here
def knapsack(items, max_weight):

    dp = [0] * (max_weight + 1)  # dp[j] 表示容量为 j 时可获得的最大价值

    for weight, value in items:
        # 倒序遍历，确保每个物品只能使用一次（0-1 背包）
        for j in range(max_weight, weight - 1, -1):
            dp[j] = max(dp[j], dp[j - weight] + value)

    return dp[max_weight]

def main():
    # 物品列表，每个元组为 (重量, 价值)
    items = [(5, 10), (4, 40), (6, 30), (3, 50)]
    max_weight = 9

    max_value = knapsack(items, max_weight)
    print(f"背包最大可容纳重量为 {max_weight}，可获得的最大价值为：{max_value}")

if __name__ == "__main__":
    main()


# your algorithm time complexity is:O(n * W)

背包最大可容纳重量为 9，可获得的最大价值为：90


## 问题 6 
给定 `n` 个非负整数表示每个宽度为 `1` 的柱子的高度图，计算按此排列的柱子，下雨之后能接多少雨水。



<div style="text-align:center;width:800px;"> <img alt="" src="./fig/hw3q6.png"> </div>

请分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：该问题可通过双指针法高效解决：使用两个指针分别从左右两端向中间移动，同时维护当前位置左侧和右侧的最大高度。每次比较左右高度，较小的一侧决定当前可能存储雨水的上限，计算该位置的可接水量后指针内移，最终累加所有位置的雨水量即为答案。


In [4]:
# add your code here
def calculate_trapped_water(heights):

    left, right = 0, len(heights) - 1  # 初始化左右指针
    max_left, max_right = 0, 0         # 当前左侧和右侧的最大高度
    total_water = 0                    # 总的接水量

    while left < right:
        if heights[left] < heights[right]:
            # 更新左侧最大高度，并计算当前位置可以接的水
            max_left = max(max_left, heights[left])
            total_water += max_left - heights[left]
            left += 1
        else:
            # 更新右侧最大高度，并计算当前位置可以接的水
            max_right = max(max_right, heights[right])
            total_water += max_right - heights[right]
            right -= 1

    return total_water

if __name__ == "__main__":
    
    print(calculate_trapped_water([0, 1, 0, 2, 1, 0, 1, 3, 2, 1, 2, 1])) 
    print(calculate_trapped_water([4, 2, 0, 3, 2, 5]))                
       

# your algorithm time complexity is: O(n)

6
9


## 问题 7 

**股票投资组合优化**： 假设你是一位投资者，想要在不同的股票中分配你的资金，以最大化你的投资回报。每只股票都有不同的预期收益率和风险。你的目标是选择一些股票，使得总投资金额不超过你的预算，并且预期收益最大化。

在这个情况下，你可以将每只股票视为一个“物品”，其重量为投资金额，价值为预期收益率。然后，你可以使用分级背包问题的方法来选择部分股票，以便在预算内获得最大的预期收益。

以下是一个简化的例子：

假设你有以下三只股票可供选择：

1. 股票 A：投资金额 5000 美元，预期收益率 10%
2. 股票 B：投资金额 3000 美元，预期收益率 8%
3. 股票 C：投资金额 2000 美元，预期收益率 12%

请设计算法找到最优投资方案，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：将每只股票的投资金额视为“重量”，其价值为 投资金额 × 收益率（即实际收益），问题转化为标准的0-1背包问题。定义 dp[j] 为预算为 j 时的最大收益，逆序遍历每只股票，更新 dp[j] = max(dp[j], dp[j - 投资金额] + 实际收益)，确保每只股票仅选一次，最终 dp[预算] 为最大总收益。



In [7]:
# add your code here
def max_investment_return(stocks, budget):

    # 将每支股票转换为 (投资金额, 投资收益)
    investments = [(amount, amount * rate) for amount, rate in stocks]

    # 初始化动态规划数组，dp[i] 表示预算为 i 时的最大收益
    dp = [0.0] * (budget + 1)

    # 遍历每个投资选项（0-1 背包，倒序遍历）
    for amount, profit in investments:
        for current_budget in range(budget, amount - 1, -1):
            # 决定是否投资当前股票
            dp[current_budget] = max(dp[current_budget], dp[current_budget - amount] + profit)

    return dp[budget]


if __name__ == "__main__":
    # 每项股票：投资金额（元） 和 收益率
    stock_options = [
        (5000, 0.10),  # 投资5000美元，回报率10%
        (3000, 0.08),  # 投资3000美元，回报率8%
        (2000, 0.12)   # 投资2000美元，回报率12%
    ]
    total_budget = 8000  # 投资总预算

    max_profit = max_investment_return(stock_options, total_budget)
    print(f"最大收益为：{max_profit:.2f} 美元")


# your algorithm time complexity is:O(n * W)

最大收益为：740.00 元


## 问题 8

给你二叉搜索树的根节点 root ，该树中的恰好两个节点的值被错误地交换。请在不改变其结构的情况下，恢复这棵树 。设计算法解决该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

e.g.:
---
1.

<div style="text-align:center;width:200px;"> <img alt="" src="./fig/hw3q9-1.jpg"> </div>
输入：root = [1,3,null,null,2]， 输出为：[3,1,null,null,2]， 

解释：3 不能是 1 的左孩子，因为 3 > 1 。交换 1 和 3 使二叉搜索树有效。

2.

 <div style="text-align:center;width:200px;"> <img alt="" src="./fig/hw3q9-1.jpg"> </div>
输入： root=[3,1,4,null,null,2]， 输出为：[2,1,4,null,null,3]

解释：2 不能在 3 的右子树中，因为 2 < 3 。交换 2 和 3 使二叉搜索树有效。

idea：二叉搜索树（BST）中序遍历应为严格递增序列。当两个节点被错误交换时，中序遍历会出现两个逆序对（或一个逆序对，若相邻节点交换）。通过中序遍历找到这两个逆序点，记录错误节点 first 和 second，最后交换它们的值即可恢复BST。


In [9]:
# add your code here
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val        # 节点值
        self.left = left      # 左子节点
        self.right = right    # 右子节点

class Solution:
    def recoverTree(self, root: TreeNode) -> None:

        self.prev = None       # 记录前一个遍历的节点
        self.first = None      # 第一个错误节点
        self.second = None     # 第二个错误节点

        self.inorder_traverse(root)

        # 交换两个错误节点的值
        if self.first and self.second:
            self.first.val, self.second.val = self.second.val, self.first.val

    # 中序遍历二叉搜索树，并识别出被错误交换的两个节点。
    def inorder_traverse(self, node: TreeNode) -> None:

        if not node:
            return

        # 递归遍历左子树
        self.inorder_traverse(node.left)

        # 检查当前节点与前一个节点的大小关系是否正确
        if self.prev and self.prev.val > node.val:
            if not self.first:
                self.first = self.prev  # 第一次发现逆序对，记录 prev
            self.second = node          # 每次发现逆序对都更新 second

        self.prev = node  # 更新 prev 节点为当前节点

        # 递归遍历右子树
        self.inorder_traverse(node.right)
        
# 从列表构建二叉树（层序），None 表示空节点
def build_tree(nodes):
 
    if not nodes:
        return None

    root = TreeNode(nodes[0])
    queue = [root]
    index = 1

    while queue and index < len(nodes):
        node = queue.pop(0)

        # 构建左子节点
        if index < len(nodes) and nodes[index] is not None:
            node.left = TreeNode(nodes[index])
            queue.append(node.left)
        index += 1

        # 构建右子节点
        if index < len(nodes) and nodes[index] is not None:
            node.right = TreeNode(nodes[index])
            queue.append(node.right)
        index += 1

    return root

# 按层打印二叉树，None 表示空节点。
# 去除尾部多余的 None
def print_tree(root):

    if not root:
        return []

    result = []
    queue = [root]

    while queue:
        node = queue.pop(0)
        if node:
            result.append(node.val)
            queue.append(node.left)
            queue.append(node.right)
        else:
            result.append(None)

    # 去掉尾部多余的 None
    while result and result[-1] is None:
        result.pop()

    return result


root = build_tree([1, 3, None, None, 2])
solution = Solution()
solution.recoverTree(root)
print(print_tree(root))  


# your algorithm time complexity is:O(n)

[3, 1, None, None, 2]


## 问题 9  

给你一个二叉树的根节点 root ，树中每个节点都存放有一个 0 到 9 之间的数字。每条从根节点到叶节点的路径都代表一个数字：

> 例如，从根节点到叶节点的路径 1 -> 2 -> 3 表示数字 123 。

设计一个算法计算从根节点到叶节点生成的所有数字之和 ，分析算法设计思路，计算时间复杂度，并基于python编程实现

叶节点:是指没有子节点的节点。

e.g.:
---
1.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q10-1.jpg"> </div>

 输入：root = [1,2,3], 输出：25

解释：从根到叶子节点路径 1->2 代表数字 12，1->3 代表数字 13，因此和为 12 + 13 = 25 。

2.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q10-2.jpg"> </div>

 输入：root = [4,9,0,5,1], 输出：1026

解释：从根到叶子节点路径 4->9->5 代表数字 495，4->9->1 代表数字 491，4->0 代表数字 40。因此和为 495 + 491 + 40 = 1026 。

idea：从根节点出发，递归遍历所有路径，维护当前路径的数值（父路径值 ×10 + 当前节点值）。当到达叶子节点时，将当前路径值累加到总和中。

In [10]:
# add your code here
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val        # 当前节点的值
        self.left = left      # 左子节点
        self.right = right    # 右子节点

class Solution:
    def sumNumbers(self, root: TreeNode) -> int:

        def dfs(node: TreeNode, current_sum: int) -> int:
            if not node:
                return 0

            # 当前路径形成的数字
            current = current_sum * 10 + node.val

            # 如果是叶子节点，返回这条路径形成的数字
            if not node.left and not node.right:
                return current

            # 否则递归计算左子树和右子树路径的数字和
            return dfs(node.left, current) + dfs(node.right, current)

        return dfs(root, 0)

def build_tree(nodes):

    if not nodes:
        return None

    root = TreeNode(nodes[0])
    queue = [root]
    index = 1

    while queue and index < len(nodes):
        node = queue.pop(0)

        # 构建左子节点
        if index < len(nodes) and nodes[index] is not None:
            node.left = TreeNode(nodes[index])
            queue.append(node.left)
        index += 1

        # 构建右子节点
        if index < len(nodes) and nodes[index] is not None:
            node.right = TreeNode(nodes[index])
            queue.append(node.right)
        index += 1

    return root

if __name__ == "__main__":
    root = build_tree([1, 2, 3])
    result = Solution().sumNumbers(root)
    print(f"从根到叶的数字之和为：{result}")



# your algorithm time complexity is:O(n)

从根到叶的数字之和为：25


## 问题 10  

给你一个二叉树的根节点 root ， 检查它是否轴对称。

1. 分析算法设计思路，计算时间复杂度，并基于python编程实现
2. \* 设计使用递归和迭代两种方法解决这个问题，计算时间复杂度，并基于python编程实现

e.g.:
---
1.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q11-1.png"> </div>

 输入：root = [1,2,2,3,4,4,3]， 输出：true

解释：二叉树 [1,2,2,3,4,4,3] 对称。

2.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q11-2.png"> </div>

 输入：root = [1,2,2,null,3,null,3]， 输出：false

idea：递归对称性检查：对称二叉树的左子树与右子树互为镜像。定义递归函数检查两节点是否满足：两节点值相等；左节点的左子树与右节点的右子树对称；左节点的右子树与右节点的左子树对称。 从根节点的左右子节点开始递归，确保整棵树对称。

In [13]:
# add your code here
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val        # 节点的值
        self.left = left      # 左子节点
        self.right = right    # 右子节点

class Solution:
    #  判断一棵二叉树是否是镜像对称的
    def isSymmetric(self, root: TreeNode) -> bool:

        if not root:
            return True
        return self.isMirror(root.left, root.right)

    # 判断两个子树是否镜像对称
    def isMirror(self, left: TreeNode, right: TreeNode) -> bool:

        # 如果都为空，说明对称
        if not left and not right:
            return True
        # 如果有一个为空，一个不为空，说明不对称
        if not left or not right:
            return False
        # 如果值不同，也不对称
        if left.val != right.val:
            return False
        # 递归比较：左子树的左 和 右子树的右；左子树的右 和 右子树的左
        return self.isMirror(left.left, right.right) and self.isMirror(left.right, right.left)

# 层序构建二叉树，None 表示空节点
def build_tree(nodes):

    if not nodes:
        return None

    root = TreeNode(nodes[0])
    queue = [root]
    index = 1

    while queue and index < len(nodes):
        node = queue.pop(0)

        # 构建左子节点
        if index < len(nodes) and nodes[index] is not None:
            node.left = TreeNode(nodes[index])
            queue.append(node.left)
        index += 1

        # 构建右子节点
        if index < len(nodes) and nodes[index] is not None:
            node.right = TreeNode(nodes[index])
            queue.append(node.right)
        index += 1

    return root

# 示例测试
if __name__ == "__main__":

    root = build_tree([1, 2, 2, 3, 4, 4, 3])
    result = Solution().isSymmetric(root)
    print(result)



# your algorithm time complexity is:O(n)

True


## 问题 11  

给你一个由 '1'（陆地）和 '0'（水）组成的的二维网格，请你计算网格中岛屿的数量。岛屿总是被水包围，并且每座岛屿只能由水平方向和/或竖直方向上相邻的陆地连接形成。

此外，你可以假设该网格的四条边均被水包围。

分析算法设计思路，计算时间复杂度，并基于python编程实现

e.g.:
---
1.
输入：grid = [

    ["1","1","1","1","0"],

    ["1","1","0","1","0"], 

    ["1","1","0","0","0"], 

    ["0","0","0","0","0"]  

]

输出：1

2.
输入：grid = [

    ["1","1","0","0","0"],

    ["1","1","0","0","0"],

    ["0","0","1","0","0"],

    ["0","0","0","1","1"]

]

输出：3

idea:使用深度优先搜索（DFS）：遍历网格，遇到陆地（'1'）时，通过DFS将与其相连的所有陆地标记为已访问（置为'0'），并计数岛屿数量。

In [14]:
# add your code here
from typing import List

class Solution:
    def numIslands(self, grid: List[List[str]]) -> int:

        if not grid or not grid[0]:
            return 0

        rows, cols = len(grid), len(grid[0])
        count = 0

        def dfs(r: int, c: int):
            # 边界检查或当前格子不是陆地就返回
            if r < 0 or c < 0 or r >= rows or c >= cols or grid[r][c] != '1':
                return
            # 将访问过的陆地标记为“0”以防止重复访问
            grid[r][c] = '0'
            # 向四个方向扩展搜索
            dfs(r + 1, c)
            dfs(r - 1, c)
            dfs(r, c + 1)
            dfs(r, c - 1)

        # 遍历整个网格
        for i in range(rows):
            for j in range(cols):
                # 每遇到一个未访问的“1”就启动一次 DFS，计数加一
                if grid[i][j] == '1':
                    dfs(i, j)
                    count += 1

        return count


if __name__ == "__main__":
    grid1 = [
        ["1", "1", "1", "1", "0"],
        ["1", "1", "0", "1", "0"],
        ["1", "1", "0", "0", "0"],
        ["0", "0", "0", "0", "0"]
    ]
    print("岛屿数量为：", Solution().numIslands(grid1)) 

    grid2 = [
        ["1", "1", "0", "0", "0"],
        ["1", "1", "0", "0", "0"],
        ["0", "0", "1", "0", "0"],
        ["0", "0", "0", "1", "1"]
    ]
    print("岛屿数量为：", Solution().numIslands(grid2))  


# your algorithm time complexity is:O(m×n)

岛屿数量为： 1
岛屿数量为： 3
